## 제2유형_연습하기_팁 예측하기(회귀)

## 데이터 분석 순서

- 1. 라이브러리 및 데이터 확인
  2. 데이터 탐색(EDA)
  3. 데이터 전처리 및 분리
  4. 모델링 및 성능평가
  5. 예측값 제출

## 1. 라이브러리 및 데이터 확인

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

df = sns.load_dataset('tips')
x = df.drop(['tip'], axis=1)
y = df['tip']

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=2023)
x_test = pd.DataFrame(x_test.reset_index())
x_train = pd.DataFrame(x_train.reset_index())
y_train = pd.DataFrame(y_train.reset_index())

x_test.rename(columns={'index':'cust_id'}, inplace=True)
x_train.rename(columns={'index':'cust_id'}, inplace=True)
y_train.columns = ['cust_id','target']

## 레스토랑의 tip 예측 문제
- 데이터의 결측치, 이상치, 변수에 대해 처리하고
- 회귀모델을 사용하여 Rsq, MSE 값을 산출하시오.

## 데이터셋 설명

- total_bill(총	청구액):	손님의	식사	총	청구액
- tip(팁):	팁의	양
- sex(성별):	손님의	성별
- smoker(흡연자):	손님의	흡연	여부("Yes"	또는	"No")
- day(요일):	식사가	이루어진	요일
- time(시간):	점심	또는	저녁	중	언제	식사가	이루어졌는지
- size(인원	수):	식사에	참석한	인원	수

## 2. 데이터 탐색(EDA)

In [2]:
# 데이터의 행/열 확인
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(195, 7)
(49, 7)
(195, 2)


In [3]:
# 초기 데이터 확인
print(x_train.head(3))
print(x_test.head(3))
print(y_train.head(3))

   cust_id  total_bill     sex smoker  day    time  size
0      158       13.39  Female     No  Sun  Dinner     2
1      186       20.90  Female    Yes  Sun  Dinner     3
2       21       20.29  Female     No  Sat  Dinner     2
   cust_id  total_bill     sex smoker  day    time  size
0      154       19.77    Male     No  Sun  Dinner     4
1        4       24.59  Female     No  Sun  Dinner     4
2       30        9.55    Male     No  Sat  Dinner     2
   cust_id  target
0      158    2.61
1      186    3.50
2       21    2.75


In [4]:
# 변수명과 데이터 타입이 매칭이 되는지, 결측치가 있는지 확인
print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   cust_id     195 non-null    int64   
 1   total_bill  195 non-null    float64 
 2   sex         195 non-null    category
 3   smoker      195 non-null    category
 4   day         195 non-null    category
 5   time        195 non-null    category
 6   size        195 non-null    int64   
dtypes: category(4), float64(1), int64(2)
memory usage: 6.0 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   cust_id     49 non-null     int64   
 1   total_bill  49 non-null     float64 
 2   sex         49 non-null     category
 3   smoker      49 non-null     category
 4   day         49 non-null     category
 5   time        49 non-null     category
 6   size   

In [5]:
# x_train과 x_test 데이터의 기초통계량을 잘 비교
print(x_train.describe().T)
print(x_test.describe().T)

            count        mean        std   min    25%     50%      75%     max
cust_id     195.0  122.056410  70.668034  0.00  59.50  121.00  182.500  243.00
total_bill  195.0   20.054667   8.961645  3.07  13.42   17.92   24.395   50.81
size        195.0    2.543590   0.942631  1.00   2.00    2.00    3.000    6.00
            count        mean        std   min    25%     50%     75%    max
cust_id      49.0  119.285714  70.918674  2.00  62.00  123.00  180.00  239.0
total_bill   49.0   18.716531   8.669864  5.75  12.74   16.66   21.01   44.3
size         49.0    2.673469   0.987162  2.00   2.00    2.00    3.00    6.0


In [6]:
# object, category 데이터도 추가 확인
# print(x_train.describe(include='object'))
# print(x_test.describe(include='object'))

print(x_train.describe(include='category').T)
print(x_test.describe(include='category').T)

       count unique     top freq
sex      195      2    Male  125
smoker   195      2      No  120
day      195      4     Sat   71
time     195      2  Dinner  142
       count unique     top freq
sex       49      2    Male   32
smoker    49      2      No   31
day       49      4     Sat   16
time      49      2  Dinner   34


In [7]:
# y데이터도 구체적으로 살펴보세요
print(y_train.head(3))

   cust_id  target
0      158    2.61
1      186    3.50
2       21    2.75


In [8]:
print(y_train.describe().T)

         count        mean        std  min   25%     50%     75%    max
cust_id  195.0  122.056410  70.668034  0.0  59.5  121.00  182.50  243.0
target   195.0    3.021692   1.402690  1.0   2.0    2.92    3.53   10.0


## 3. 데이터 전처리 및 분리

### 1)결측치, 2)이상치, 3)변수 처리하기

In [9]:
# 결측치 확인
print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

cust_id       0
total_bill    0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64
cust_id       0
total_bill    0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64
cust_id    0
target     0
dtype: int64


In [10]:
# 결측치 제거
# df.dropna()
# df.dropna().shape

In [11]:
# 결측치 대체(평균값, 중앙값, 최빈값)
# ** 주의사항 : train 데이터의 중앙값으로 test데이터도 변경해줘야 함 **
# 연속형 변수 : 중앙값, 평균값
# df['변수명'].median()
# df['변수명'].mean()

# 범주형 변수 : 최빈값
# df['변수명'].mode()

# df['변수명'] = df['변수명'].fillna(대체할 값)

In [12]:
# 이상치 대체
# df['변수명'] = np.where(df['변수명']>5, 대체할 값, df['변수명'])

In [13]:
# 변수처리
# 불필요한 변수 제거
# df = df.drop(columns=['변수1','변수2'])
# df = df.drop(['변수1','변수2'], axis=1)

# 필요시 변수 추가(파생변수 생성)
# df['파생변수명'] = df['A']*df['B']

# 원핫인코딩(가변수 처리)
#x_train = pd.get_dummies(x_train)
#x_test = pd.get_dummies(x_test)

In [14]:
# 변수처리
cust_id = x_test['cust_id'].copy()

# 각 데이터에서 cust_id 변수 제거
x_train = x_train.drop(['cust_id'], axis=1)
x_test = x_test.drop(columns=['cust_id'])

In [15]:
# 변수처리(원핫인코딩)
print(x_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  195 non-null    float64 
 1   sex         195 non-null    category
 2   smoker      195 non-null    category
 3   day         195 non-null    category
 4   time        195 non-null    category
 5   size        195 non-null    int64   
dtypes: category(4), float64(1), int64(1)
memory usage: 4.5 KB
None


In [16]:
# 변수처리(원핫인코딩)
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)
print(x_train.info())
print(x_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   total_bill   195 non-null    float64
 1   size         195 non-null    int64  
 2   sex_Male     195 non-null    bool   
 3   sex_Female   195 non-null    bool   
 4   smoker_Yes   195 non-null    bool   
 5   smoker_No    195 non-null    bool   
 6   day_Thur     195 non-null    bool   
 7   day_Fri      195 non-null    bool   
 8   day_Sat      195 non-null    bool   
 9   day_Sun      195 non-null    bool   
 10  time_Lunch   195 non-null    bool   
 11  time_Dinner  195 non-null    bool   
dtypes: bool(10), float64(1), int64(1)
memory usage: 5.1 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   total_bill   49 non-null     float64
 1   size    

### 데이터 분리

In [17]:
# 데이터를 훈련 세트와 검증용 세트로 분할 (80% 훈련, 20% 검증용)
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                  y_train['target'],
                                                  test_size=0.2,
                                                  random_state=2023)
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(156, 12)
(39, 12)
(156,)
(39,)


## 4. 모델링 및 성능평가

In [18]:
# 랜덤포레스트 모델 사용 (참고 : 분류모델은 RandomForestClassifier)
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=2023)
model.fit(x_train, y_train)

RandomForestRegressor(random_state=2023)

In [19]:
# 모델을 사용하여 테스트 데이터 예측
y_pred = model.predict(x_val)

In [20]:
# 모델 성능 평가 (R-squared, MSE 등)
from sklearn.metrics import r2_score, mean_squared_error
r2 = r2_score(y_val, y_pred) # (실제값, 예측값)
mse = mean_squared_error(y_val, y_pred) # (실제값, 예측값)

In [21]:
# MSE
print(mse)

1.0860209841025643


In [22]:
# RMSE
rmse = mse**0.5
print(rmse)

1.042123305613383


In [23]:
# Rsq
print(r2)

0.6214234662421672


## 5. 예측값 제출

In [24]:
# pd.DataFrame({'cust_id':cust_id, 'target':y_result}).to_csv('수험번호.csv', index=False)

y_result = model.predict(x_test)
result = pd.DataFrame({'cust_id':cust_id, 'target':y_result})
print(result[:5])

   cust_id  target
0      154  3.1114
1        4  2.8938
2       30  1.7715
3       75  1.6389
4       33  3.1873


In [26]:
# 데이터를 저장한다음 불러와서 제대로 제출했는지 확인해보자
pd.DataFrame({'result':y_result}).to_csv('0010.csv', index=False)


In [27]:
df2 = pd.read_csv('0010.csv')
print(df2.head())

   result
0  3.1114
1  2.8938
2  1.7715
3  1.6389
4  3.1873
